In [3]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint, solve_ivp
from lmfit import minimize, Parameters, report_fit, Minimizer
import matplotlib.pyplot as plt

In [6]:
file_path2 = (r'C:\Users\camil\Documents\IMT\2A\Stage_ingénierie\Notebook\Kon_18-04-2024.xlsx')
# Charger toutes les feuilles de calcul dans un dictionnaire de DataFrames
xls_1 = pd.ExcelFile(file_path2)

# Afficher les noms des feuilles de calcul disponibles
print("Feuilles de calcul disponibles :", xls_1.sheet_names)

# Lire une feuille spécifique 
df_1 = pd.read_excel(file_path2, sheet_name='Feuille 1')

Feuilles de calcul disponibles : ['Feuille 1', 'Feuille 2', 'Feuille 3']


In [7]:
t = df_1['Time points (min)'].values
dt = df_1['Luminescence ligne 1'].values
dt2 = df_1['Luminescence ligne 2'].values
dt3 = df_1['Luminescence ligne 3'].values
dt4 = df_1['Luminescence ligne 4'].values
dt5 = df_1['Luminescence ligne 5'].values
dt6 = df_1['Luminescence ligne 6'].values
dt7 = df_1['Luminescence ligne 7'].values
dt8 = df_1['Luminescence ligne 8'].values

In [8]:
def check_mass_conservation(A, B, A0, B0, rtol=1e-9):
    if not np.isclose(A0 + B0, A + B + (A0 + B0 - A - B), rtol=rtol):
        raise ValueError(f"Mass not conserved. A0 + B0 = {A0 + B0}, A + B + C = {A + B + (A0 + B0 - A - B)}")

In [9]:

def model(t, y, params):
    A0 = params['A0'].value
    B0 = params['B0'].value
    A, B = y *np.array([[A0], [B0]])
    C = A0 + B0 - A - B  # Calculate C from A and B
    kon = params['kon'].value
    koff = params['koff'].value

    check_mass_conservation(A, B, A0, B0)

    dAdt = (-kon * A * B + koff * C)/A0
    dBdt = (-kon * A * B + koff * C)/B0

    return [dAdt, dBdt]

In [10]:
def solve_model(t, params):
    A0 = params['A0'].value
    B0 = params['B0'].value
    y0 = [1, 1]

    sol = solve_ivp(lambda t, y: model(t, y, params), [t[0], t[-1]], y0, t_eval=t, method='BDF', rtol=1e-9, atol=1e-9)
    
    if not sol.success:
        raise RuntimeError(f"solve_ivp failed: {sol.message}")
    
    A, B = sol.y *np.array([[A0], [B0]])
    C = A0 + B0 - A - B
    return np.column_stack((A, B, C))


In [11]:
def residual(params, t, data):
    try:
         
        if params['kon'].value * params['A0'].value * params['B0'].value > 1e10:
            return np.full_like(data, np.inf)
        
        model_values = solve_model(t, params)[:, 2]  # Concentration du complexe C
        alpha = params['alpha'].value
        beta = params['beta'].value
        luminescence = alpha * model_values + beta
        return (luminescence - data) / np.sqrt(data)  # Weighted residuals
    
    except Exception as e:
        print(f"Error in residual function: {str(e)}")
        return np.full_like(data, np.inf)

In [12]:
params = Parameters()
params.add('alpha', value=1e3, min=0, max=1e5)
params.add('beta', value=1, min=0, max=1e2)
params.add('kon', value=1e5, min=1e3, max=1e7, vary=True)
params.add('koff', value=1e-2, min=1e-4, max=1e2, vary=True)
params.add('A0', value=2e-5, min=1e-7, max=1e-3, vary=True)
params.add('B0', value=3e-6, min=1e-7, max=1e-3, vary=True)

In [13]:
try:
    result = minimize(residual, params, args=(t, dt), method='differential_evolution', max_nfev=2000)
    print("\nRésultats de l'ajustement:")
    print(report_fit(result))
    
    # Tracer les résultats
    fitted_values = solve_model(t, result.params)[:, 2]
    luminescence = result.params['alpha'].value * fitted_values + result.params['beta'].value
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, dt, 'o', label='Données')
    plt.plot(t, luminescence, '-', label='Modèle ajusté')
    plt.legend()
    plt.xlabel('Temps (min)')
    plt.ylabel('Luminescence')
    plt.title('Ajustement du modèle aux données')
    plt.show()
    
    # Afficher les paramètres optimisés
    print("\nParamètres optimisés:")
    for name, param in result.params.items():
        print(f"{name}: {param.value:.4e} +/- {param.stderr:.4e}")
    
except Exception as e:
    print(f"Une erreur s'est produite pendant l'optimisation : {str(e)}")

Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error in residual function: The truth va

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
